In [ ]:
import requests
import pandas as pd

def format_wca_time(cs):
    if cs == -1: return "DNF"
    if cs == -2: return "DNS"
    if cs is None or cs <= 0: return ""
    hundredths = cs % 100
    total_seconds = cs // 100
    seconds = total_seconds % 60
    minutes = total_seconds // 60
    if minutes > 0:
        return f"{minutes}:{seconds:02d}.{hundredths:02d}"
    return f"{seconds}.{hundredths:02d}"

# 1. Fetch and Parse
person_id = "2016LOPE37"
url = f"https://raw.githubusercontent.com/robiningelbrecht/wca-rest-api/master/api/persons/{person_id}.json"
data = requests.get(url).json()

rows = []
# We reverse the keys to process the oldest competitions first
comp_ids = list(data["results"].keys())[::-1]

for comp_id in comp_ids:
    events = data["results"][comp_id]
    for event_id, rounds in events.items():
        # Rounds are usually listed Final -> First, so reverse those too
        for r in reversed(rounds):
            solves = r.get("solves", [])
            rows.append({
                "Competition": comp_id,
                "Event": event_id,
                "Round": r.get("round"),
                "best_cs": r.get("best"),
                "avg_cs": r.get("average"),
                "time1": solves[0] if len(solves) > 0 else None,
                "time2": solves[1] if len(solves) > 1 else None,
                "time3": solves[2] if len(solves) > 2 else None,
                "time4": solves[3] if len(solves) > 3 else None,
                "time5": solves[4] if len(solves) > 4 else None,
            })

df = pd.DataFrame(rows)

# 2. Logic for PRs
# DNF (-1) and DNS (-2) must be treated as infinity for comparison
def clean_for_min(val):
    return float('inf') if val <= 0 else val

# Track current bests per event
running_best_single = {}
running_best_avg = {}
pr_labels = []

for idx, row in df.iterrows():
    e = row["Event"]
    s = clean_for_min(row["best_cs"])
    a = clean_for_min(row["avg_cs"])
    
    is_s_pr = s <= running_best_single.get(e, float('inf')) and s != float('inf')
    is_a_pr = a <= running_best_avg.get(e, float('inf')) and a != float('inf')
    
    # Update running bests
    if is_s_pr: running_best_single[e] = s
    if is_a_pr: running_best_avg[e] = a
    
    # Assign labels
    if is_s_pr and is_a_pr: pr_labels.append("sin+avg")
    elif is_s_pr: pr_labels.append("single")
    elif is_a_pr: pr_labels.append("average")
    else: pr_labels.append(None)

df["pr"] = pr_labels

# 3. Final Formatting
time_cols = ["time1", "time2", "time3", "time4", "time5", "best_cs", "avg_cs"]
for col in time_cols:
    df[col] = df[col].apply(format_wca_time)

# Rename columns for final output
df = df.rename(columns={"best_cs": "Best", "avg_cs": "Average"})

# Reverse back to show newest results at the top
df = df.iloc[::-1].reset_index(drop=True)

print(df[["Competition", "Event", "Round", "Best", "Average", "pr"]].head(20))

# export to excel
df.to_excel("results"+str(person_id)+".xlsx", index=False)

In [2]:
pip install pandas requests openpyxl

  Using cached pandas-2.3.3-cp311-cp311-win_amd64.whl.metadata (19 kB)
  Using cached requests-2.32.5-py3-none-any.whl.metadata (4.9 kB)
  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached charset_normalizer-3.4.4-cp311-cp311-win_amd64.whl.metadata (38 kB)
  Using cached idna-3.11-py3-none-any.whl.metadata (8.4 kB)
Using cached pandas-2.3.3-cp311-cp311-win_amd64.whl (11.3 MB)
Using cached requests-2.32.5-py3-none-any.whl (64 kB)
   ---------------------------------------- 0.0/250.9 kB ? eta -:--:--
   ---- ----------------------------------- 30.7/250.9 kB 1.4 MB/s eta 0:00:01
   ---------------------------------------- 250.9/250.9 kB 3.9 MB/s eta 0:00:00
   ---------------------------------------- 0.0/159.4 kB ? eta -:--:--
   ---------------------------------------- 159.4/159.4 kB 9.3 MB/s eta 0:00:00
Using cached charset_normalizer-3.4.4-cp311-cp311-win_amd64.whl (106 kB)
Using cached idna-3.11-py3-none-any.whl (71 kB)
   ----------------------------------


[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: C:\Users\ruben\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [4]:
# contar numero de prs
pr_count = df['pr'].value_counts()
# si es de sin+avg cuenta como 2 prs
pr_count = pr_count.get('single', 0) + pr_count.get('average', 0) + pr_count.get('sin+avg', 0) * 2
print("\nNúmero de PRs obtenidos:")
print(pr_count)

# desglosa por categoría: 3x3x3, 2x2x2, 4x4x4, etc.
pr_by_event = df[df['pr'].notnull()]['Event'].value_counts()
print("\nPRs por categoría:")
print(pr_by_event)



Número de PRs obtenidos:
306

PRs por categoría:
Event
333      23
222      23
sq1      23
444      22
555      21
pyram    21
clock    20
minx     18
skewb    17
333oh    16
666       8
333fm     7
777       5
333ft     1
Name: count, dtype: int64


# Competiciones organizadas

In [6]:
import requests
import pandas as pd

name_to_search = "Rubén López de Juan"
all_competitions = []

print(f"Buscando competiciones organizadas por: {name_to_search}...")

# Iteramos por las 18 páginas de la API
for i in range(1, 19):
    url = f"https://raw.githubusercontent.com/robiningelbrecht/wca-rest-api/master/api/competitions-page-{i}.json"
    response = requests.get(url)
    
    # Verificamos que la página exista antes de procesar
    if response.status_code == 200:
        data = response.json()
        for competition in data['items']:
            # Extraemos los nombres de los organizadores
            organisers_names = [org["name"] for org in competition.get("organisers", [])]
            
            if name_to_search in organisers_names:
                # Creamos el diccionario con la estructura deseada
                comp_data = {
                    "Nombre": competition.get("name"),
                    "id": competition.get("id"),
                    "city": competition.get("city"),
                    "country": competition.get("country"),
                    "date_start": competition.get("date", {}).get("from"),
                    "date_end": competition.get("date", {}).get("till"),
                    "no_days": competition.get("date", {}).get("numberOfDays")
                }
                all_competitions.append(comp_data)

# Creamos el DataFrame
df = pd.DataFrame(all_competitions)

# Añadimos la columna "Numero" (empezando en 1)
df.insert(0, "Numero", range(1, len(df) + 1))

# Mostramos el resultado
print(f"\nSe han encontrado {len(df)} competiciones en "+str(df['country'].nunique())+" regiones")


# Si quieres guardarlo a Excel o CSV:
df.to_excel("competiciones_organizadas.xlsx", index=False)

Buscando competiciones organizadas por: Rubén López de Juan...

Se han encontrado 42 competiciones en 4 regiones


In [ ]:
def get_wcaid_prs(wca_id):
    # request to wca api https://www.worldcubeassociation.org/api/v0/persons/{WCA_ID}
    url = f"https://www.worldcubeassociation.org/api/v0/persons/{wca_id}"
    data = requests.get(url).json()
    rows = []
    print(data)

In [2]:
import requests

wca_id = "2016LOPE37"
url = f"https://www.worldcubeassociation.org/api/v0/persons/{wca_id}"
data = requests.get(url).json()
print(data)

{'person': {'name': 'Rubén López de Juan', 'gender': 'm', 'url': 'https://www.worldcubeassociation.org/persons/2016LOPE37', 'country': {'id': 'Spain', 'name': 'Spain', 'continent_id': '_Europe', 'iso2': 'ES'}, 'location': 'Spain', 'region_id': 17, 'delegate_status': 'delegate', 'email': 'rjuan@worldcubeassociation.org', 'class': 'person', 'teams': [{'id': 1365, 'friendly_id': 'wrc', 'leader': False, 'senior_member': False, 'name': 'Rubén López de Juan', 'wca_id': '2016LOPE37', 'avatar': {'id': 79344, 'status': 'approved', 'thumbnail_crop_x': 223, 'thumbnail_crop_y': 0, 'thumbnail_crop_w': 2026, 'thumbnail_crop_h': 2026, 'url': 'https://avatars.worldcubeassociation.org/9h1ald0pb5t1mmc7nds12idkovpo', 'thumb_url': 'https://avatars.worldcubeassociation.org/zgpvws1bqybgq3dmtnltx5kpyi8e', 'is_default': False, 'can_edit_thumbnail': True}}, {'id': 1975, 'friendly_id': 'wmct', 'leader': True, 'senior_member': False, 'name': 'Rubén López de Juan', 'wca_id': '2016LOPE37', 'avatar': {'id': 79344, 